In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# Load and clean the data
games = pd.read_csv("games_full.csv")
games = games.dropna(subset=["title"])
all_titles = sorted(games["title"].drop_duplicates().tolist())

# Combine relevant text columns
text_cols = ["genres", "supported_languages", "developers", "publishers"]
games[text_cols] = games[text_cols].fillna("")
games["text_features"] = games[text_cols].agg(" ".join, axis=1)

#### WIDGETS ####
search_box = widgets.Combobox(
    placeholder="Type a game title here...",
    options=all_titles,
    description="Search:",
    ensure_option=True,
    layout=widgets.Layout(width="70%"),
)

add_button = widgets.Button(description="Add Game")
reset_button = widgets.Button(description="Reset", button_style="danger")
confirm_button = widgets.Button(description="Get Recommendations", button_style="success", layout=widgets.Layout(width="70%"))

liked_list = widgets.SelectMultiple(
    options=[], description="Liked Games:", rows=8, layout=widgets.Layout(width="70%")
)

output = widgets.Output()

#### CALLBACKS ####
def on_add_click(b):
    game = search_box.value.strip()
    if game in all_titles and game not in liked_list.options:
        liked_list.options = list(liked_list.options) + [game]
        search_box.value = ""

def on_reset_click(b):
    liked_list.options = []
    search_box.value = ""
    with output:
        output.clear_output()

def on_confirm_click(b):
    with output:
        output.clear_output()
        liked_titles = list(liked_list.options)
        
        if not liked_titles:
            print("Please add at least one game to get recommendations.")
            return

        print("You selected:")
        for title in liked_titles:
            print(f"• {title}")
        
        # TF-IDF Vectorizer
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(games["text_features"])

        # Indices of liked games
        liked_indices = games[games["title"].isin(liked_titles)].index

        # Cosine similarity between liked games and all games
        similarity_matrix = cosine_similarity(X[liked_indices], X)

        # Average similarity score across all liked games
        avg_similarity = similarity_matrix.mean(axis=0)

        # Add similarity scores to DataFrame
        games["similarity_score"] = avg_similarity

        # Exclude liked games from recommendations
        recommendations = games[~games["title"].isin(liked_titles)]
        recommendations = recommendations.sort_values("similarity_score", ascending=False)

        print("\n🎮 Top 10 Game Recommendations 🎮")
        for i, row in recommendations[["title", "similarity_score"]].head(10).iterrows():
            print(f"• {row['title']} (Similarity: {row['similarity_score']:.2f})")

        # 🔎 Sanity Check – show reason for top recommendation
        print("\nWhy was this recommended?")

        top_game = recommendations.iloc[0]
        print(f"\nTop Recommendation: {top_game['title']}")
        print(f"Genres: {top_game['genres']}")
        print(f"Developers: {top_game['developers']}")
        print(f"Publishers: {top_game['publishers']}")
        print(f"Languages: {top_game['supported_languages']}")

        liked_game = games[games["title"] == liked_titles[0]].iloc[0]
        print("\n🔍 Comparing with liked game:")
        print(f"Liked Game: {liked_game['title']}")
        print(f"Genres: {liked_game['genres']}")
        print(f"Developers: {liked_game['developers']}")
        print(f"Publishers: {liked_game['publishers']}")
        print(f"Languages: {liked_game['supported_languages']}")


#### HOOK UP CALLBACKS ####
try:
    add_button._click_callbacks.callbacks.clear()
    reset_button._click_callbacks.callbacks.clear()
    confirm_button._click_callbacks.callbacks.clear()
except Exception:
    pass

add_button.on_click(on_add_click)
reset_button.on_click(on_reset_click)
confirm_button.on_click(on_confirm_click)

#### DISPLAY UI ####
ui = widgets.VBox(
    [
        widgets.HBox([search_box, add_button, reset_button]),
        liked_list,
        confirm_button,
        output,
    ]
)

display(ui)

In [5]:
# Check why a recommended game was picked
print(games.loc[games['title'] == 'Your Recommended Game Title', 'text_features'].values[0])


IndexError: index 0 is out of bounds for axis 0 with size 0